# NYISO Energy Data — Exploratory Data Analysis
**Project 01: Data Pipeline & EDA**

This notebook walks through:
1. Fetching real NYISO data via the public CSV API
2. Cleaning and structuring the time series
3. Exploratory analysis: load patterns, price distributions, fuel mix
4. Visual insights that will feed into the forecasting model (Project 02)

> **Note:** If you're offline or want to run quickly without hitting NYISO servers, set `USE_SYNTHETIC = True` in the Config cell. The synthetic data mimics real NYISO patterns for development/demo purposes.

---

## 0. Setup & Config

In [16]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ── Config ────────────────────────────────────────────────────────────────────
USE_SYNTHETIC = False     # Set True to skip API calls and use generated data
START_DATE    = datetime(2020, 1, 1)
END_DATE      = datetime(2025, 12, 31)   # 3 months is enough for good EDA
DATA_DIR      = '../data/raw'
PROC_DIR      = '../data/processed'

print('Setup complete.')

Setup complete.


## 1. Data Ingestion

We pull three datasets from NYISO's public CSV API:
- **Actual Load** (`pal`) — hourly MW consumption by zone
- **Day-Ahead LMP** (`damlbmp`) — locational marginal prices ($/MWh) by zone
- **Fuel Mix** (`rtfuelmix`) — generation MW by fuel type (gas, nuclear, hydro, wind, etc.)

In [17]:
from nyiso_client import fetch_date_range
from clean import clean, make_hourly, pivot_zones
import os

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)

if USE_SYNTHETIC:
    # ── Synthetic data generator (mimics NYISO patterns) ─────────────────────
    # Useful for development without hitting live servers
    print('Generating synthetic NYISO-like data...')
    from generate_synthetic import make_synthetic_data
    df_load, df_lmp, df_fuel = make_synthetic_data(START_DATE, END_DATE)
else:
    # ── Live NYISO API fetch ──────────────────────────────────────────────────
    print('Fetching live data from NYISO...')
    print('This will take ~30-60 seconds for a 3-month range.\n')
    
    raw_load = fetch_date_range('load_actual', START_DATE, END_DATE, save_dir=DATA_DIR)
    raw_lmp  = fetch_date_range('lmp_dayahead', START_DATE, END_DATE, save_dir=DATA_DIR)
    raw_fuel = fetch_date_range('fuel_mix', START_DATE, END_DATE, save_dir=DATA_DIR)

    df_load = clean(raw_load, 'load_actual')
    df_lmp  = clean(raw_lmp,  'lmp_dayahead')
    df_fuel = clean(raw_fuel, 'fuel_mix')

print(f'Load data:    {df_load.shape}')
print(f'LMP data:     {df_lmp.shape}')
print(f'Fuel mix:     {df_fuel.shape}')

2026-02-22 23:45:01,117 [INFO]   Loading from cache: ../data/raw\load_actual_202001.parquet
2026-02-22 23:45:01,142 [INFO]   Loading from cache: ../data/raw\load_actual_202002.parquet
2026-02-22 23:45:01,162 [INFO]   Loading from cache: ../data/raw\load_actual_202003.parquet
2026-02-22 23:45:01,182 [INFO]   Loading from cache: ../data/raw\load_actual_202004.parquet
2026-02-22 23:45:01,204 [INFO]   Loading from cache: ../data/raw\load_actual_202005.parquet
2026-02-22 23:45:01,228 [INFO]   Loading from cache: ../data/raw\load_actual_202006.parquet
2026-02-22 23:45:01,250 [INFO]   Loading from cache: ../data/raw\load_actual_202007.parquet
2026-02-22 23:45:01,276 [INFO]   Loading from cache: ../data/raw\load_actual_202008.parquet


Fetching live data from NYISO...
This will take ~30-60 seconds for a 3-month range.



2026-02-22 23:45:01,298 [INFO]   Loading from cache: ../data/raw\load_actual_202009.parquet
2026-02-22 23:45:01,321 [INFO]   Loading from cache: ../data/raw\load_actual_202010.parquet
2026-02-22 23:45:01,342 [INFO]   Loading from cache: ../data/raw\load_actual_202011.parquet
2026-02-22 23:45:01,365 [INFO]   Loading from cache: ../data/raw\load_actual_202012.parquet
2026-02-22 23:45:01,387 [INFO]   Loading from cache: ../data/raw\load_actual_202101.parquet
2026-02-22 23:45:01,414 [INFO]   Loading from cache: ../data/raw\load_actual_202102.parquet
2026-02-22 23:45:01,435 [INFO]   Loading from cache: ../data/raw\load_actual_202103.parquet
2026-02-22 23:45:01,456 [INFO]   Loading from cache: ../data/raw\load_actual_202104.parquet
2026-02-22 23:45:01,475 [INFO]   Loading from cache: ../data/raw\load_actual_202105.parquet
2026-02-22 23:45:01,497 [INFO]   Loading from cache: ../data/raw\load_actual_202106.parquet
2026-02-22 23:45:01,518 [INFO]   Loading from cache: ../data/raw\load_actual_202

2026-02-22 23:45:03,191 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202202.parquet
2026-02-22 23:45:03,198 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202203.parquet
2026-02-22 23:45:03,205 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202204.parquet
2026-02-22 23:45:03,211 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202205.parquet
2026-02-22 23:45:03,217 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202206.parquet
2026-02-22 23:45:03,224 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202207.parquet
2026-02-22 23:45:03,232 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202208.parquet
2026-02-22 23:45:03,238 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202209.parquet
2026-02-22 23:45:03,244 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202210.parquet
2026-02-22 23:45:03,251 [INFO]   Loading from cache: ../data/raw\lmp_dayahead_202211.parquet
2026-02-22 23:45:03,257 [INFO]   Loading from cache: ../data/raw\lmp_d

2026-02-22 23:45:04,256 [INFO]   Loading from cache: ../data/raw\fuel_mix_202308.parquet
2026-02-22 23:45:04,275 [INFO]   Loading from cache: ../data/raw\fuel_mix_202309.parquet
2026-02-22 23:45:04,292 [INFO]   Loading from cache: ../data/raw\fuel_mix_202310.parquet
2026-02-22 23:45:04,311 [INFO]   Loading from cache: ../data/raw\fuel_mix_202311.parquet
2026-02-22 23:45:04,329 [INFO]   Loading from cache: ../data/raw\fuel_mix_202312.parquet
2026-02-22 23:45:04,348 [INFO]   Loading from cache: ../data/raw\fuel_mix_202401.parquet
2026-02-22 23:45:04,366 [INFO]   Loading from cache: ../data/raw\fuel_mix_202402.parquet
2026-02-22 23:45:04,384 [INFO]   Loading from cache: ../data/raw\fuel_mix_202403.parquet
2026-02-22 23:45:04,404 [INFO]   Loading from cache: ../data/raw\fuel_mix_202404.parquet
2026-02-22 23:45:04,423 [INFO]   Loading from cache: ../data/raw\fuel_mix_202405.parquet
2026-02-22 23:45:04,439 [INFO]   Loading from cache: ../data/raw\fuel_mix_202406.parquet
2026-02-22 23:45:04,4

Load data:    (7063683, 6)
LMP data:     (788940, 9)
Fuel mix:     (4507671, 5)


In [18]:
# Quick sanity check on the load data
df_load.head(12)

,timestamp,Time Zone,zone,PTID,load_mw,load_mw_outlier
0,2020-01-01 00:00:00-05:00,NaN,CAPITL,61757,1208.4,False
1,2020-01-01 00:00:00-05:00,NaN,CENTRL,61754,1668.6,False
2,2020-01-01 00:00:00-05:00,NaN,DUNWOD,61760,573.0,False
3,2020-01-01 00:00:00-05:00,NaN,GENESE,61753,1008.3,False
4,2020-01-01 00:00:00-05:00,NaN,HUD VL,61758,978.4,False
5,2020-01-01 00:00:00-05:00,NaN,LONGIL,61762,2048.4,False
6,2020-01-01 00:00:00-05:00,NaN,MHK VL,61756,809.5,False
7,2020-01-01 00:00:00-05:00,NaN,MILLWD,61759,318.7,False
8,2020-01-01 00:00:00-05:00,NaN,N.Y.C.,61761,5012.3,False
9,2020-01-01 00:00:00-05:00,NaN,NORTH,61755,565.4,False


In [19]:
# Check date coverage and missing timestamps
def check_coverage(df, name):
    ts = df['timestamp']
    # Only check nulls on actual data columns, not metadata like Time Zone
    data_cols = [c for c in df.columns if c not in ('timestamp', 'Time Zone')]
    null_count = df[data_cols].isnull().sum().sum()
    
    print(f'\n{name}')
    print(f'  Range:   {ts.min()} → {ts.max()}')
    print(f'  Rows:    {len(df):,}')
    print(f'  Nulls:   {null_count}')
    
check_coverage(df_load, 'Actual Load')
check_coverage(df_lmp,  'Day-Ahead LMP')
check_coverage(df_fuel, 'Fuel Mix')


Actual Load
  Range:   2020-01-01 00:00:00-05:00 → 2025-12-31 23:55:00-05:00
  Rows:    7,063,683
  Nulls:   77

Day-Ahead LMP
  Range:   2020-01-01 00:00:00-05:00 → 2025-12-31 23:00:00-05:00
  Rows:    788,940
  Nulls:   0

Fuel Mix
  Range:   2020-01-01 00:05:00-05:00 → 2026-01-01 00:00:00-05:00
  Rows:    4,507,671
  Nulls:   0


## 2. Load Analysis

### 2a. System-wide load over time

NYISO's total system load is the sum across all 11 zones. Let's look at the shape of demand across our date range.

In [20]:
# Aggregate to total system load per timestamp
df_system = (
    df_load
    .groupby('timestamp')['load_mw']
    .sum()
    .reset_index()
    .rename(columns={'load_mw': 'total_load_mw'})
)

fig = px.line(
    df_system,
    x='timestamp', y='total_load_mw',
    title='NYISO System-Wide Actual Load (MW)',
    labels={'total_load_mw': 'Load (MW)', 'timestamp': ''},
    template='plotly_dark'
)
fig.update_traces(line_color='#00d4ff', line_width=1)
fig.update_layout(height=350)
fig.show()

### 2b. Load by zone — which zones drive peak demand?

In [21]:
# Average load per zone
zone_avg = (
    df_load
    .groupby('zone')['load_mw']
    .mean()
    .sort_values(ascending=True)
    .reset_index()
)

fig = px.bar(
    zone_avg,
    x='load_mw', y='zone',
    orientation='h',
    title='Average Load by NYISO Zone (MW)',
    labels={'load_mw': 'Avg Load (MW)', 'zone': ''},
    color='load_mw',
    color_continuous_scale='Blues',
    template='plotly_dark'
)
fig.update_layout(height=400, coloraxis_showscale=False)
fig.show()



N.Y.C. and LONGIL dominate — they typically account for ~60% of total load.

### 2c. Hourly load profile — the "duck curve" pattern

The average load by hour of day reveals the classic electricity demand shape: overnight trough, morning ramp, mid-day plateau, evening peak.

In [22]:
df_system['hour']       = df_system['timestamp'].dt.hour
df_system['day_of_week']= df_system['timestamp'].dt.day_name()
df_system['month']      = df_system['timestamp'].dt.month
df_system['is_weekend'] = df_system['timestamp'].dt.dayofweek >= 5

hourly_profile = (
    df_system
    .groupby(['hour', 'is_weekend'])['total_load_mw']
    .mean()
    .reset_index()
)
hourly_profile['day_type'] = hourly_profile['is_weekend'].map({True: 'Weekend', False: 'Weekday'})

fig = px.line(
    hourly_profile,
    x='hour', y='total_load_mw',
    color='day_type',
    title='Average Hourly Load Profile: Weekday vs Weekend',
    labels={'total_load_mw': 'Avg Load (MW)', 'hour': 'Hour of Day'},
    color_discrete_map={'Weekday': '#00d4ff', 'Weekend': '#ff6b35'},
    template='plotly_dark',
    markers=True
)
fig.update_layout(height=350)
fig.show()

Key insight: Weekday load is ~10-15% higher than weekends.

Both show a morning ramp (~6AM) and evening peak (~7-8PM) — critical for forecasting.

### 2d. Load heatmap — day × hour

This is one of the most informative views for energy analysts. Each cell is the average load for that hour-of-day × day-of-week combination.

In [23]:
dow_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

heatmap_data = (
    df_system
    .groupby(['day_of_week', 'hour'])['total_load_mw']
    .mean()
    .reset_index()
    .pivot(index='day_of_week', columns='hour', values='total_load_mw')
    .reindex(dow_order)
)

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=[f'{h:02d}:00' for h in range(24)],
    y=dow_order,
    colorscale='Blues',
    colorbar=dict(title='MW')
))
fig.update_layout(
    title='Average Load Heatmap: Day of Week × Hour of Day',
    template='plotly_dark',
    height=350,
    xaxis_title='Hour',
    yaxis_title=''
)
fig.show()

## 3. LMP Price Analysis

Locational Marginal Prices (LMP) represent the cost of electricity at each grid location. They're the primary price signal in wholesale energy markets.

High LMP periods are when avoided energy has the most financial value

In [24]:
# Pivot to wide format: one column per zone
lmp_wide = pivot_zones(df_lmp, 'lmp_total')

# NYC vs. rest of state — usually NYC is highest due to congestion
zones_to_plot = ['N.Y.C.', 'LONGIL', 'CAPITL', 'WEST']
available = [z for z in zones_to_plot if z in lmp_wide.columns]

fig = px.line(
    lmp_wide,
    x='timestamp',
    y=available,
    title='Day-Ahead LMP by Zone ($/MWh)',
    labels={'value': '$/MWh', 'timestamp': '', 'variable': 'Zone'},
    template='plotly_dark'
)
fig.update_layout(height=380)
fig.show()

In [25]:
# Price distribution — looking for spikes
fig = px.box(
    df_lmp[df_lmp['zone'].isin(['N.Y.C.', 'LONGIL', 'CAPITL', 'WEST', 'NORTH'])],
    x='zone', y='lmp_total',
    title='LMP Distribution by Zone (outliers = price spikes)',
    labels={'lmp_total': '$/MWh', 'zone': 'Zone'},
    color='zone',
    template='plotly_dark'
)
fig.update_layout(height=380, showlegend=False)
fig.show()

# Summary stats
print('\nLMP Summary by Zone ($/MWh):')
print(
    df_lmp.groupby('zone')['lmp_total']
    .agg(['mean','median','std','max'])
    .round(2)
    .sort_values('mean', ascending=False)
)


LMP Summary by Zone ($/MWh):
         mean  median    std     max
zone                                
LONGIL  55.05   41.64  42.09  569.81
CAPITL  50.01   36.35  41.95  550.69
NPX     49.22   36.30  39.11  405.10
N.Y.C.  47.84   36.36  36.84  425.00
DUNWOD  46.87   35.70  35.45  398.16
MILLWD  46.76   35.61  35.30  396.70
HUD VL  46.08   35.15  34.72  394.83
PJM     39.38   31.89  27.56  369.83
MHK VL  38.00   29.69  30.36  385.64
CENTRL  36.88   28.94  28.83  362.47
WEST    36.20   28.52  28.11  653.41
GENESE  35.61   27.99  27.70  350.64
H Q     33.79   26.81  28.23  367.25
O H     33.77   26.94  25.46  334.67
NORTH   32.72   25.82  28.81  373.13


## 4. Fuel Mix Analysis

What generates the electricity? NYISO's fuel mix shows the real-time contribution of each source — this is the "carbon content" signal and is increasingly important for ESG reporting and demand-side programs.

In [26]:
# Average generation by fuel type
fuel_avg = (
    df_fuel
    .groupby('fuel_type')['gen_mw']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

fig = px.pie(
    fuel_avg,
    values='gen_mw',
    names='fuel_type',
    title='Average Generation Mix by Fuel Type',
    template='plotly_dark',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(height=420)
fig.show()

In [27]:
# Fuel mix over time — stacked area chart
fuel_hourly = (
    df_fuel
    .set_index('timestamp')
    .groupby('fuel_type')
    .resample('D')['gen_mw']  # daily average to keep chart readable
    .mean()
    .reset_index()
)

fig = px.area(
    fuel_hourly,
    x='timestamp', y='gen_mw',
    color='fuel_type',
    title='Daily Average Generation Mix Over Time',
    labels={'gen_mw': 'Generation (MW)', 'timestamp': '', 'fuel_type': 'Fuel'},
    template='plotly_dark'
)
fig.update_layout(height=400)
fig.show()

In [28]:
# ── Validate fuel mix claims from actual data ─────────────────────────────────

# Join LMP and fuel mix on timestamp
df_lmp_hourly = df_lmp.copy()
df_lmp_hourly['ts_hour'] = df_lmp_hourly['timestamp'].dt.floor('h')

df_fuel_hourly = df_fuel.copy()
df_fuel_hourly['ts_hour'] = df_fuel_hourly['timestamp'].dt.floor('h')
df_fuel_hourly = df_fuel_hourly.groupby(['ts_hour', 'fuel_type'])['gen_mw'].mean().reset_index()

# Pivot fuel mix to wide format
fuel_wide = df_fuel_hourly.pivot_table(
    index='ts_hour',
    columns='fuel_type',
    values='gen_mw',
    aggfunc='mean'
).reset_index()

# Get system-wide average LMP by hour
lmp_avg = df_lmp_hourly.groupby('ts_hour')['lmp_total'].mean().reset_index()

# Merge
df_merged = lmp_avg.merge(fuel_wide, on='ts_hour', how='inner')

# ── What fuels generate most during low vs high price hours ──────────────────
low_price  = df_merged[df_merged['lmp_total'] < df_merged['lmp_total'].quantile(0.25)]
high_price = df_merged[df_merged['lmp_total'] > df_merged['lmp_total'].quantile(0.75)]

fuel_cols = [c for c in fuel_wide.columns if c != 'ts_hour']

print("=== Average Generation by Fuel Type ===\n")
print(f"{'Fuel':<20} {'Low Price (MW)':>15} {'High Price (MW)':>16} {'Difference':>12}")
print("-" * 65)

for fuel in sorted(fuel_cols):
    if fuel in df_merged.columns:
        low_gen  = low_price[fuel].mean()
        high_gen = high_price[fuel].mean()
        diff     = high_gen - low_gen
        print(f"{fuel:<20} {low_gen:>15.1f} {high_gen:>16.1f} {diff:>12.1f}")

# ── What % of hours does each fuel dominate generation ───────────────────────
print("\n=== Dominant Generation Fuel by Price Quartile ===")
print("\nTop generating fuel during LOW price hours:")
print(low_price[fuel_cols].mean().sort_values(ascending=False).head(3))

print("\nTop generating fuel during HIGH price hours:")
print(high_price[fuel_cols].mean().sort_values(ascending=False).head(3))

# ── Correlation between each fuel and LMP ─────────────────────────────────────
print("\n=== Correlation between Fuel Generation and LMP ===")
correlations = df_merged[fuel_cols + ['lmp_total']].corr()['lmp_total'].drop('lmp_total')
print(correlations.sort_values(ascending=False))

=== Average Generation by Fuel Type ===

Fuel                  Low Price (MW)  High Price (MW)   Difference
-----------------------------------------------------------------
Dual Fuel                     2833.3           4982.0       2148.7
Hydro                         2988.5           3210.8        222.3
Natural Gas                   2592.0           4081.8       1489.8
Nuclear                       3864.3           3206.7       -657.6
Other Fossil Fuels              19.2             46.7         27.6
Other Renewables               261.8            279.0         17.2
Wind                           589.8            600.0         10.2

=== Dominant Generation Fuel by Price Quartile ===

Top generating fuel during LOW price hours:
Nuclear      3864.279799
Hydro        2988.454319
Dual Fuel    2833.278253
dtype: float64

Top generating fuel during HIGH price hours:
Dual Fuel      4981.994839
Natural Gas    4081.797690
Hydro          3210.782999
dtype: float64

=== Correlation between Fue

## 5. Cross-Dataset Insights

### 5a. Load vs. Price correlation
Does higher load always mean higher prices? Not always — congestion and fuel costs matter too.

In [29]:
# Merge system load with NYC LMP for scatter analysis
nyc_lmp = df_lmp[df_lmp['zone'] == 'N.Y.C.'][['timestamp','lmp_total']]

df_merged = pd.merge(
    df_system[['timestamp','total_load_mw','hour','is_weekend']],
    nyc_lmp,
    on='timestamp',
    how='inner'
)

df_merged['period'] = df_merged['is_weekend'].map({True: 'Weekend', False: 'Weekday'})

fig = px.scatter(
    df_merged.sample(min(2000, len(df_merged))),  # sample for performance
    x='total_load_mw',
    y='lmp_total',
    color='hour',
    symbol='period',
    title='System Load vs. NYC LMP — colored by Hour of Day',
    labels={'total_load_mw': 'System Load (MW)', 'lmp_total': 'NYC LMP ($/MWh)'},
    template='plotly_dark',
    color_continuous_scale='Viridis',
    opacity=0.6
)
fig.update_layout(height=450)
fig.show()

corr = df_merged[['total_load_mw','lmp_total']].corr().iloc[0,1]
print(f'\nLoad–Price Pearson correlation: {corr:.3f}')
print('High correlation = price is load-driven. Low = congestion/fuel dominate.')


Load–Price Pearson correlation: 0.455
High correlation = price is load-driven. Low = congestion/fuel dominate.


## 6. Save Processed Data

Save clean files for use in Project 02 (forecasting). Using parquet for speed and size efficiency.

In [30]:
import os
os.makedirs(PROC_DIR, exist_ok=True)

df_load.to_parquet(f'{PROC_DIR}/load_actual.parquet', index=False)
df_lmp.to_parquet(f'{PROC_DIR}/lmp_dayahead.parquet', index=False)
df_fuel.to_parquet(f'{PROC_DIR}/fuel_mix.parquet', index=False)
df_system.to_parquet(f'{PROC_DIR}/system_load.parquet', index=False)

print('Saved to data/processed/:')
for f in os.listdir(PROC_DIR):
    size = os.path.getsize(f'{PROC_DIR}/{f}') / 1024
    print(f'  {f}  ({size:.1f} KB)')

Saved to data/processed/:
  fuel_mix.parquet  (19247.6 KB)
  lmp_dayahead.parquet  (4410.1 KB)
  load_actual.parquet  (54886.7 KB)
  system_load.parquet  (9547.0 KB)


## 7. Key Takeaways

After this EDA, we've established:

1. **Load follows strong time patterns** — hour of day, weekday vs weekend, and seasonal effects are all present. These become features in the forecasting model.

2. **NYC and Long Island dominate demand** — ~60% of system load. Zone-level models may outperform system-level ones.

3. **LMP is volatile and zone-dependent** — congestion causes prices to diverge significantly across zones. Price spike detection is a natural follow-on project.

4. **Natural gas is the marginal fuel** — setting prices most of the time. Renewable penetration (wind, hydro) provides price suppression during off-peak periods.